In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
feature_x_sample = pd.read_csv("../input/feature_x_sample.tsv", sep="\t", index_col=0)

comparison_x_sample = pd.read_csv(
    "../output/comparison_x_sample.tsv", sep="\t", index_col=0
)

sets = kraft.gmt.read_many(kraft.path.list_("../input/feature_set/"))

set_x_sample = pd.read_csv("../output/set_x_sample.tsv", sep="\t", index_col=0)

In [ ]:
for comparison, groups in comparison_x_sample.iterrows():

    groups.dropna(inplace=True)

    min_group_size = np.unique(groups.to_numpy(), return_counts=True)[1].min()

    function, group_size = "get_ic", 2

    if min_group_size < group_size:

        continue

    directory_path = os.path.join("../output/expand_set", comparison, "")

    kraft.path.path(directory_path)

    scores = pd.concat(
        (
            pd.read_csv(
                os.path.join(
                    "../output/find_differentially_expressed_feature",
                    kraft.path.clean(comparison),
                    "all_{}".format(function),
                    "scores.tsv",
                ),
                sep="\t",
                index_col=0,
            ),
            pd.read_csv(
                os.path.join(
                    "../output/find_differentially_expressed_set",
                    kraft.path.clean(comparison),
                    "all_{}".format(function),
                    "scores.tsv",
                ),
                sep="\t",
                index_col=0,
            ),
        )
    ).drop_duplicates()

    for set_name in SETTING["sets_to_peek"]:

        _x_sample = pd.concat(
            (
                feature_x_sample.reindex(labels=sets[set_name]),
                set_x_sample.reindex(labels=(set_name,)),
            )
        )

        kraft.function_heat_map.make(
            groups,
            _x_sample,
            function=None,
            scores=scores,
            n_extreme=None,
            se_data_type="binary",
            plot_std=SETTING["plot_std"],
            title=set_name,
            directory_path=directory_path,
        )